In [2]:
# Initialize & Establish MongoDB Connection
from pymongo import MongoClient
from pandas import DataFrame
import pandas as pd
import urllib.parse
%matplotlib inline
from matplotlib_venn import venn3
from matplotlib_venn import venn3_unweighted
import matplotlib.pyplot as plt
import numpy as np
import json
import ast

def batched(cur, batch_size):
    batch = []
    for doc in cur:
        batch.append(doc)
        if batch and not len(batch) % batch_size:
            yield batch
            batch = []
    if batch:  
        yield batch
        
def process_query(col,query):
    cur = db[col].aggregate(query,allowDiskUse = True)

    df = pd.DataFrame()
    for batch in batched(cur, 10000):
        dfb = pd.DataFrame(list(batch))
        df = pd.concat([df, dfb])
        
    return df

mongo_host = "10.0.0.1"
mongo_port = 27017

mongo_user = "root"
mongo_pass = ""  

auth_db = "admin" 

client = MongoClient(f'mongodb://{mongo_user}:{urllib.parse.quote_plus(mongo_pass)}@{mongo_host}:{mongo_port}/{auth_db}') 
db = client.android

colist = db.list_collection_names()
print(colist)

['apps', 'manifest', 'new_man']


In [3]:
col = 'apps'
query = [{ "$sort": {"InstallsMax": -1}},{ "$project": {"_id": '$ID', "Name": '$Title', "Downloads": '$InstallsMax'}}]
apps_ranked = process_query(col,query)

In [4]:
apps_top_50k = apps_ranked.head(50000)
apps_top_50k.to_csv("/apps-top-50k.csv", sep=',')

In [5]:
print(apps_top_50k)

                                           _id  \
0                       com.google.android.gms   
1                   com.google.android.youtube   
2                       com.google.android.tts   
3                           com.android.chrome   
4                 com.google.android.apps.maps   
...                                        ...   
9995  co.pamobile.gamestudio.jurassicdinosaurs   
9996                 com.midinero.mobile.myapp   
9997      com.ma.chained.cars.stunts.freegames   
9998           com.cb.myvacation.realadventure   
9999            com.laurencedawson.reddit_sync   

                                Name    Downloads  
0               Google Play services  16086100890  
1                            YouTube  13553071138  
2          Speech Services by Google  12956902114  
3       Google Chrome: Fast & Secure  12788965530  
4                        Google Maps  12788268330  
...                              ...          ...  
9995  Dino Sim Dinosaur City Rampag